In [2]:
from typing import List
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import primes

In [3]:
# Generate Primes

# 'P' is a list of prime numbers.

RANGE_END = 1000
nums = range(2, RANGE_END)
P = primes.gen_primes_sieve_to(max(nums))
print (f"{len(P)} primes, max: {P[-1]}")


168 primes, max: 997


In [4]:
# Factor each prime.
# 'factors' a list, each element is a tuple of:
#		( prime, [list of factors])
#
factors = []

for n in nums:
	if not n in P:
		factors.append( (n, primes.find_factors_fwdall(n)) )


In [5]:
#
# Test each factoring
# AND
# convert factors to the prime powers.



for q in factors:
	#print(f"{q[0]} factors {q[1]}")

	# Test the factoring.
	prod = 1
	for p in q[1]:
		# Each factor must be a prime
		if not p in P:					
			print (f"Not prime {p}")
		prod *= p
	# product of the factors must be the number.
	if prod != q[0]:
		print (f"  product of factors is not correct:  {prod}")

	# Find factors using a different algorithm.
	# Must be the same.
	revfactros = primes.find_factors_rev(q[0], P)
	revfactros.sort()
	if revfactros != q[1]:
		print (f"  reverse factors are not correct:  {revfactros}")



In [6]:
#
# Build two DataFrames.
# powersdf has
#	columns - the prime numbers from 'P'
#	index - the composit (non prime) numbers
# 	Each row is the factoring expressed as the power of each prime
#
# plotabledf is a plotable representation.  Each row is one plotable point with the columns:
# 	Number - the composit number
#	Factor - one of the composit number's prime factors
#	Power - That prime's power - how many times it appears as a factor.
#
#powers = []
powersdf = pd.DataFrame(columns=P, index=[q[0] for q in factors])	# and then a dataframe
plotablepoints = []
for q in factors:
	p = primes.primes_to_powers(q[1], P)
	#powers.append(p)
	powersdf.loc[q[0]] = p
	for i in range(len(p)):
		if p[i] > 0:
			plotablepoints.append([q[0], P[i], p[i]])

plotabledf = pd.DataFrame(plotablepoints, columns=["Number", "Factor", "Power"])
plotablepoints = None
p = None

# Convert zero to NA
	nadf = powersdf.applymap(lambda f: f if f > 0 else pd.NA)
	print (nadf)

In [7]:

#
# Take the powers python array and make factors pandas dataframe
# where columns are prime numbers, rows are integers, and cell values
# are the power of prime.  product of primes raised to powers are the 
# index integer.
#
powersnp = np.array(powers)										# Make a numpy array
df = pd.DataFrame(powers, columns=P, index=[q[0] for q in factors])	# and then a dataframe
print (df.shape)
Pr = P.copy()													# Create a copy of the list of primes
Pr.reverse()													# and reverse that 
for p in Pr:													# so we can search from largest to smallest
	# print (f"{p} {df[p].sum()}")		
	if df[p].sum() == 0:										# If the column contains no powers
		df.drop(p, axis='columns', inplace=True)				#   we drop that column
	else:
		break													#   otherwise, we have reached the largest prime used and exit the loop
print (df.shape)
df.to_csv("local-primefactors.csv")



NameError: name 'powers' is not defined

In [ ]:
#
# Convert a dataframe into structure that can be 3D plotted.
#

def plottable_3d_info(df: pd.DataFrame):
    """
    Transform Pandas data into a format that's compatible with
    Matplotlib's surface and wireframe plotting.
    """
    index = df.index
    columns = df.columns

    x, y = np.meshgrid(np.arange(len(columns)), np.arange(len(index)))
    z = np.array([[df[c][i] for c in columns] for i in index])
    
    xticks = dict(ticks=np.arange(len(columns)), labels=columns)
    yticks = dict(ticks=np.arange(len(index)), labels=index)
    
    return x, y, z, xticks, yticks


In [ ]:
# Use the function above to arrange data for plotting.
x, y, z, xticks, yticks = plottable_3d_info(df)

In [ ]:
plotabledf['Power'].max()

In [ ]:
### Set up axes and put data on the surface.
fig = plt.figure(figsize=(50,50))
axes = fig.add_subplot(projection='3d')
#axes.scatter(plotabledf["Number"], plotabledf["Factor"], plotabledf["Power"], c=plotabledf["Factor"])
axes.stem3D(plotabledf["Factor"], plotabledf["Number"], plotabledf["Power"])

# How to reverse an axes:  axes.set_xlim(plotabledf['Number'].max(), 0)
axes.set_xlabel('Factor')
axes.set_ylabel('Number')
axes.set_zlabel('Power')
axes.set_zlim3d(bottom=0)
axes.set_zticks(np.arange(plotabledf['Power'].max()+1))			# Z tick every number.

plt.show()

In [ ]:
### Set up axes and put data on the surface.
plt.close()
fig = plt.figure(figsize=(50,50))
axes = fig.add_subplot()
#axes.scatter(plotabledf["Number"], plotabledf["Factor"], plotabledf["Power"], c=plotabledf["Factor"])
axes.scatter(plotabledf["Number"], plotabledf["Factor"], c=plotabledf["Power"])

# How to reverse an axes:  axes.set_xlim(plotabledf['Number'].max(), 0)
axes.set_xlabel('Number')
axes.set_ylabel('Factor')

plt.colorbar()

plt.show()

In [ ]:

fig = px.scatter_3d(x = plotabledf["Number"], y=plotabledf["Factor"], z=plotabledf["Power"])
fig.update_layout(width=1000, height=1000)
fig.update_traces(marker_size=2, marker_color='green')
fig.show()
fig.write_html("jj.html")